In [ ]:
%load_ext autoreload
%autoreload 2
import os
import matplotlib.pyplot as plt
import seaborn as sns
from os.path import join
from tqdm import tqdm
import pandas as pd
import sys
from typing import List
import numpy as np
import joblib
from pprint import pprint
from math import ceil
import imodelsx.util
import sasc.viz
import pickle as pkl
import json
from copy import deepcopy
from numpy.linalg import norm
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, normalize
from sklearn.cluster import AgglomerativeClustering
from PIL import Image
import img2pdf

from scipy.sparse import csr_matrix

from sasc.config import CACHE_DIR, RESULTS_DIR, cache_ngrams_dir, regions_idxs_dir, FMRI_DIR, SAVE_DIR_FMRI
import sasc.modules.fmri_module
from imodels import StableClustering
ngrams_list = joblib.load(join(cache_ngrams_dir, 'fmri_UTS02_ngrams.pkl')) # ngrams are same for both models

subject = 'S02'
k_values = np.unique(np.logspace(0.5, 3, 50, dtype=int))
# subject = 'S03'
# rois_dict = joblib.load(join(regions_idxs_dir, f'rois_{subject}.jbl'))
# sorted(rois_dict.keys())
# if subject == 'S03':
#     rois_dict['OPA'] = rois_dict['TOS']
SUBJECT_VOX_COUNTS_DICT = {
    'S02': 94251,
}
# suffix_setting = ''
suffix_setting = '_filt=0.15'
# suffix_setting = '_rj_custom'

In [ ]:
# load pfc stuff
pfc = joblib.load(join(FMRI_DIR, 'brain_tune/voxel_neighbors_and_pcs/',
                  'PFC_spotlights.jbl')).astype(bool)
data = np.load(
    join(FMRI_DIR, 'brain_tune/voxel_neighbors_and_pcs/', 'connectivity.npz'))
connectivity = csr_matrix(
    (data['data'], data['indices'], data['indptr']), shape=data['shape'])

In [ ]:
mod = sasc.modules.fmri_module.fMRIModule(
    subject=f"UT{subject}",
    # checkpoint="facebook/opt-30b",
    checkpoint="huggyllama/llama-30b",
    init_model=False,
    restrict_weights=False,
)
weights_arr_full = mod.weights
# weights_arr_full has shape (num_delays x num_linear_coefs, num_voxels)

if suffix_setting == '_filt=0.15':
    print(pfc.sum())
    pfc[mod.corrs < 0.15] = False
    print('filtered to ', pfc.sum())

    # histogram, with dashes for percentiles
    # plt.hist(mod.corrs[pfc])
    # for q in np.arange(10, 100, 10):
    #     perc = np.percentile(mod.corrs[pfc], q)
    #     plt.axvline(perc, linestyle='--', color='k')

    #     # annotate with percentile and value
    #     plt.text(np.percentile(mod.corrs[pfc], q),
    #              0, f'{q}th ({perc:0.3f})', rotation=90, va='bottom')
    # plt.xlabel('Test corr')

# mean over delays
weights_arr_to_cluster = weights_arr_full.reshape(
    4, -1, weights_arr_full.shape[1]).mean(axis=0)
# weights_arr_to_cluster.shape is (num_linear_coefs, num_voxels)

# only keep PFC voxels
weights_arr_to_cluster_pfc = weights_arr_to_cluster[:, pfc]
# weights_arr_to_cluster_pfc.shape is (num_linear_coefs, num_pfc_voxels)

In [ ]:
# pca
num_pcs = 20
pca = PCA(n_components=num_pcs, whiten=True)

pca.fit(normalize(weights_arr_to_cluster_pfc.T))
# pca.fit(normalize(weights_arr_to_cluster.T))
# pca.components_.shape is (num_pcs, num_linear_coefs)

pc_coefs_per_voxel = pca.transform(normalize(weights_arr_to_cluster.T))
# pc_coefs_per_voxel.shape is (num_voxels, num_pcs)

In [ ]:
# # save pcs
# for pc_num in range(num_pcs):
#     flatmap = pc_coefs_per_voxel[:, pc_num]
#     # flatmap = np.zeros(weights_arr_full.shape[1])
#     # flatmap[pfc] = pc_coefs_per_voxel[:, pc_num]
#     sasc.viz._save_flatmap(
#         flatmap, 'S02', fname_save=f'pca_images/pca_{pc_num+1}.png')

# # read all plots and save as subplots on the same page
# fig, axs = plt.subplots(ceil(num_pcs/3), 3, figsize=(15, 5))
# axs = axs.ravel()
# for i in range(num_pcs):
#     axs[i].imshow(Image.open(f'./pca_{i+1}.png'))
#     axs[i].axis('off')
#     axs[i].set_title(f'PC {i + 1}')
# plt.savefig('pca_subplots.png')

In [ ]:
pc_coefs_per_pfc_voxel = pc_coefs_per_voxel[pfc]
connectivity_pfc = connectivity[pfc][:, pfc]

# run clustering

In [ ]:
if suffix_setting == '_rj_custom':
    clusters = joblib.load('all_new_rois_UTS02_gemv_semmod.jbl')
    cluster_arr = np.zeros_like(clusters[0])
    for i, cluster in enumerate(clusters):
        cluster_arr[cluster.astype(bool)] = i + 1
    # decrease all vals by 1 (to avoid the "0" cluster)
    cluster_arr -= 1

    cluster_assignments_dict = {len(clusters): cluster_arr}
    k_values = [len(clusters)]

else:
    cluster_assignments_dict = {}
    for num_clusters in tqdm(k_values):
        clustering = AgglomerativeClustering(
            n_clusters=num_clusters,
            connectivity=connectivity_pfc,
            linkage='ward',
        )

        clustering.fit(pc_coefs_per_pfc_voxel)
        cluster_assignments = clustering.labels_
        cluster_assignments_dict[num_clusters] = deepcopy(cluster_assignments)

        # visualize clusters
        for i in range(min(20, num_clusters)):
            # flatmap = pc_coefs_per_voxel[:, pc_num]
            flatmap = np.zeros(weights_arr_full.shape[1])
            flatmap[pfc] = (cluster_assignments == i).astype(int)
            sasc.viz._save_flatmap(
                flatmap, 'S02', fname_save=f'clusters/{i+1}.png')

        # # read all plots and save as subplots on the same page
        # C = 4
        # R = ceil(min(20, num_clusters)/C)
        # fig, axs = plt.subplots(R, C, figsize=(C * 4, R * 2))
        # axs = axs.ravel()
        # for i in range(min(20, num_clusters)):
        #     axs[i].imshow(Image.open(f'clusters/{i+1}.png'))
        #     axs[i].axis('off')
        #     axs[i].set_title(f'{i + 1}')
        # plt.savefig(f'clusters/cluster_aggregated_{num_clusters}.png')
    joblib.dump(cluster_assignments_dict, join(
        cache_ngrams_dir, f'cluster_assignments_dict_{subject}{suffix_setting}.jbl'))

In [ ]:
# cluster_assignments_dict = joblib.load(join(
#     cache_ngrams_dir, f'cluster_assignments_dict_{subject}{suffix_setting}.jbl'))

# Use clustering to get predictions etc.
(This is the slowest part, involves a massive matrix multiply)

In [ ]:
for embs_fname, checkpoint, out_suffix in zip(
    ['fmri_embs.pkl', 'fmri_embs_llama.pkl'],
    ['facebook/opt-30b', 'huggyllama/llama-30b'],
    ['_opt', '_llama'],
):
    print(f'Running for {embs_fname}')
    embs = joblib.load(join(cache_ngrams_dir, embs_fname))
    # embs = joblib.load(join(cache_ngrams_dir, 'fmri_embs_llama.pkl'))
    mod = sasc.modules.fmri_module.fMRIModule(
        subject=f"UT{subject}",
        checkpoint=checkpoint,
        init_model=False,
        restrict_weights=False,
    )
    voxel_preds = mod(embs=embs, return_all=True)

    print('Saving outputs for clusters')
    for k in tqdm(k_values):
        cluster_assignments_arr = cluster_assignments_dict[k]
        outputs_dict = {
            k_: np.nanmean(voxel_preds[:,
                                       np.where(
                                           cluster_assignments_arr == k_)[0]
                                       ], axis=1)
            for k_ in range(k)
        }
        joblib.dump(outputs_dict, join(
            cache_ngrams_dir, f'pfc_clusters_ngram_outputs_dict_{k}_{subject}{out_suffix}{suffix_setting}.pkl'))

### Compute similarities between clusters

In [ ]:
def nancorr(x, y):
    # remove nans
    mask = ~np.isnan(x) & ~np.isnan(y)
    return np.corrcoef(x[mask], y[mask])[0, 1]


cluster_corrs_list = []
for k in tqdm(k_values):
    # for k in [3]:
    outputs_dict_opt = joblib.load(join(
        cache_ngrams_dir, f'pfc_clusters_ngram_outputs_dict_{k}_{subject}_opt{suffix_setting}.pkl'))
    outputs_dict_llama = joblib.load(join(
        cache_ngrams_dir, f'pfc_clusters_ngram_outputs_dict_{k}_{subject}_llama{suffix_setting}.pkl'))

    cluster_corrs = [nancorr(outputs_dict_opt[k], outputs_dict_llama[k])
                     for k in outputs_dict_opt]
    cluster_corrs_list.append(cluster_corrs)
    # print(
    # f'k = {k}: {np.nanmean(cluster_corrs):.2f} +/- {np.nanstd(cluster_corrs):.2f}')
# plt.plot(k_values, [np.nanmean(x) for x in cluster_corrs_list], '.-')

In [ ]:
if not suffix_setting == '_rj_custom':
    cluster_assignments_dict = joblib.load(join(
        cache_ngrams_dir, f'cluster_assignments_dict_{subject}{suffix_setting}.jbl'))
cluster_sizes_list = [
    np.unique(
        cluster_assignments_dict[k][cluster_assignments_dict[k] >= 0], return_counts=True)[1]
    for k in cluster_assignments_dict
]
df_sizes = pd.DataFrame([list(x) for x in cluster_sizes_list], index=k_values)
df_sizes = df_sizes.stack().reset_index().set_index('level_0')

df = pd.DataFrame([list(x) for x in cluster_corrs_list], index=k_values)
df = df.stack().reset_index().set_index('level_0')
df['size'] = df_sizes[0].values
df['size>100'] = df['size'] > 100
df['size_str'] = df['size'].apply(
    lambda x: '<10' if x < 10 else '10-50' if x < 50 else '50-100' if x < 100 else '>100')
df.index.names = ['num_clusters']
df = df.rename(columns={'level_1': 'cluster_idx', 0: 'corr'}).reset_index()

# select some clusters
# num_clusters_selected = 123
# num_clusters_selected = 287
# num_clusters_selected = 316

if suffix_setting == '_rj_custom':
    d = df
    num_clusters_selected = 9
else:
    num_clusters_selected = 1000
    d = df[df['num_clusters'] == num_clusters_selected]

# d = d[d['size'] >= 10]
d_selected = d.sort_values('corr', ascending=False).head(200)

# add indexes voxels
idxs_vox_list = []
for i, row in tqdm(d_selected.iterrows()):
    cluster_idx = row['cluster_idx']
    flatmap = np.zeros(SUBJECT_VOX_COUNTS_DICT[subject])
    # idxs_pfc_filt = np.where(
    # cluster_assignments_dict[num_clusters_selected] == cluster_idx)[0]
    idxs_pfc_filt = cluster_assignments_dict[num_clusters_selected] == cluster_idx
    idxs = np.zeros(SUBJECT_VOX_COUNTS_DICT[subject])
    pfc_filt = pfc.copy()
    pfc_filt[mod.corrs < 0.15] = False
    idxs[pfc_filt] = idxs_pfc_filt
    idxs_vox_list.append(np.where(idxs)[0])
d_selected['idxs_vox'] = idxs_vox_list
d_selected['corr_test'] = d_selected['idxs_vox'].apply(
    lambda x: mod.corrs[x].mean())

In [ ]:
# make wide figure
plt.figure(figsize=(15, 5))

# color with discrete colormap
sns.pointplot(data=df, x='num_clusters', y='corr', errorbar=None,
              color='black', markers='.', alpha=0.5)
ax = sns.stripplot(data=df, x='num_clusters', y='corr', jitter=0.2,
                   alpha=1, size=3, hue='size_str')
ax.get_legend().set_title('Voxel count in the cluster')

plt.xlabel('k')
plt.ylabel('Stability score (per cluster)')
# plt.axhline(0, color='gray', linewidth=0.5)
plt.grid()

# rotate xlabels
plt.xticks(rotation=45)
plt.show()

In [ ]:
for i, row in tqdm(d_selected.iterrows()):
    flatmap = np.zeros(SUBJECT_VOX_COUNTS_DICT[subject])
    flatmap[pfc][row['idxs_vox']] = 1
    sasc.viz._save_flatmap(
        flatmap, 'S02', fname_save=f'clusters_selected/{i+1}.png')


# read all plots and save as subplots on the same page
C = 3
R = ceil(len(d_selected)/C)
fig, axs = plt.subplots(R, C, figsize=(C * 4, R * 2))
axs = axs.ravel()
j = 0
for i, row in d_selected.iterrows():
    axs[j].imshow(Image.open(f'clusters_selected/{i+1}.png'))
    axs[j].axis('off')
    axs[j].set_title(f'stab {row["corr"]:.2f}, {row["size"]:.0f} voxels')
    j += 1
plt.savefig(f'clusters_selected{suffix_setting}.png')

### Generate explanations

In [ ]:
outputs_dict = joblib.load(
    join(cache_ngrams_dir, f'pfc_clusters_ngram_outputs_dict_{num_clusters_selected}_{subject}_opt{suffix_setting}.pkl'))
df_opt = pd.DataFrame(outputs_dict, index=ngrams_list)
outputs_dict = joblib.load(
    join(cache_ngrams_dir, f'pfc_clusters_ngram_outputs_dict_{num_clusters_selected}_{subject}_llama{suffix_setting}.pkl'))
df_llama = pd.DataFrame(outputs_dict, index=ngrams_list)
df = df_opt + df_llama

In [ ]:
ascending = False  # should be false to get driving ngrams
# top_ngrams_dict = {}
# for k in df.columns:
# top_ngrams_dict[k] = df.sort_values(
# k, ascending=ascending).index[:100].tolist()
# if k in ROIS_LOC:

# top_ngrams_dict[k + '_only'] = df.sort_values(
# k + '_only', ascending=ascending).index[:100].tolist()
# top_ngrams_df = pd.DataFrame(top_ngrams_dict)
d_selected['top_ngrams'] = d_selected['cluster_idx'].apply(
    lambda x: df.loc[:, x].sort_values(ascending=False).index[:100].tolist())
# top_ngrams_df.to_csv(f'top_ngrams_by_pfc_cluster_{subject}.csv')
# with pd.option_context('display.max_rows', None):
# rois = ['RSC', 'OPA', 'PPA', 'IPS', 'pSTS', 'sPMv',
# 'EBA', 'OFA'] + ['RSC_only', 'OPA_only', 'PPA_only']
# rois = [r for r in rois if not r == 'pSTS']  # never localized pSTS in S03
# display(top_ngrams_df[rois])
d_selected.to_pickle(
    f'top_clusters_by_pfc_cluster_{subject}{suffix_setting}.pkl')

In [ ]:
# # generate explanations
# d_selected = pd.read_pickle(
#     f'top_clusters_by_pfc_cluster_{subject}{suffix_setting}.pkl')
# gpt4 = imodelsx.llm.get_llm('gpt-4')
# explanations = []
# prompt_template = '''Here is a list of phrases:
# {s}

# What is a common theme among these phrases? Return only a concise phrase.'''
# for i, row in d_selected.iterrows():
#     top_ngrams = row['top_ngrams']
#     s = '- ' + '\n- '.join(top_ngrams[:60])
#     prompt = prompt_template.format(s=s)
#     # print(prompt)
#     # explanations.append(gpt4(prompt))
# # joblib.dump(explanations, f'explanations_by_pfc_cluster_{subject}.jbl')

In [ ]:
EXPLANATIONS_ALREADY_RAN_S02_QA = [
    'abstract descriptions',
    'numbers',
    'specific objects or items',
    'industry or profession',
    'sensory experiences',
    'negations',
    'time',
    'personal reflections or thoughts',
    'opinions or judgments',
    'technical or specialized terminology',
    'personal values or beliefs',
    'planning or organizing',
    'proper nouns',
    'physical actions',
    'personal or interactions interactions',
    'relationships between people',
    'cultural references',
    'communication',
    'dialogue',
    'locations',
    'measurements',
]

EXPLANATIONS_ALREADY_RAN_S02_VOXEL = [
    'surprise',
    'laughter',
    'moments',
    'rejection',
    'emotion',
    'negativity',
    'emotional expression',
    'communication',
    'death',
    'birthdays',
    'physical injury or trauma',
    'hair and clothing',
    'food preparation',
    'time',
    'measurements',
    'directions',
    'locations',
]

EXPLANATIONS_ALREADY_RAN_S02_ROIS = [
]

In [ ]:
explanations = joblib.load(
    f'explanations_by_pfc_cluster_{subject}{suffix_setting}.jbl')
# d_selected['explanation'] = explanations
d_selected.insert(0, 'explanation', explanations)

In [ ]:
d_out = (d_selected
         .rename(columns={'corr': 'stability score'})
         .drop(
             columns=['cluster_idx', 'num_clusters', 'size>100', 'size_str'])
         .round(3)
         .infer_objects()
         )

In [ ]:
d_out.to_csv(f'top_clusters_by_pfc_cluster_{subject}{suffix_setting}.csv')

In [ ]:
d_selected

### S03 Export selected rois to pkl

In [ ]:
rois = ['RSC', 'OPA', 'PPA', 'IPS', 'sPMv', 'EBA', 'OFA'] + \
    ['RSC_only', 'OPA_only', 'PPA_only']  # all but 'pSTS'
# pprint({k: explanations[k] for k in rois})
explanations_clean = {
    'EBA': 'Body parts',
    'IPS': 'Descriptive elements of scenes or objects',
    # OFA differs from UTS02 (which was "'Personal growth and reflection',")
    'OFA': 'Conversational transitions',
    'OPA': 'Direction and location descriptions',
    # OPA_only differs from UTS02 (which was 'Spatial positioning and directions')
    'OPA_only': 'Self-reflection and growth',
    'PPA': 'Scenes and settings',
    'PPA_only': 'Unappetizing foods',
    'RSC': 'Travel and location names',
    'RSC_only': 'Location names',
    # sPMv differs from UTS02 (which was 'Time and Numbers')
    'sPMv': 'Dialogue and responses',
}
explanation_avoid_suffixes = {
    'EBA': ' Avoid mentioning any locations.',
    'IPS': ' Avoid mentioning any locations.',
    'OFA': ' Avoid mentioning any locations.',
    'OPA': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'OPA_only': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'PPA': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'PPA_only': ' Avoid mentioning any specific location names (like "New York" or "Europe").',
    'RSC': '',
    'RSC_only': '',
    'sPMv': ' Avoid mentioning any locations.'
}
for roi in rois:
    print(f'"{roi}":', str(
        top_ngrams_df[roi.replace('1', '').replace('2', '')].iloc[:50].values.tolist()) + ', ')
top_ngrams_clean = {
    "RSC": ['was led upstairs', 'onto the subway', 'to the hallway', 'drove to washington', 'back through london', 'and darted downstairs', 'past the offices', 'long hallway toward', 'down the sidewalk', 'back in manhattan', 'reached the interstate', 'just blocks away', 'drove from vermont', 'was standing outside', 'to a courtyard', 'in the alley', 'up the coast', 'from my dorm', 'in the courtyard', 'in central park', 'i walk outside', 'here in manhattan', 'darted downstairs', 'facing the beach', 'walk through downtown', 'wander the hallways', 'i ran downstairs', 'down the hall', "'m standing outside", 'off into vancouver', 'through the streets', 'sitting in indianapolis', 'on sixth avenue', 'i go upstairs', 'across the street', 'arrived in indianapolis', 'we were downtown'],
    "OPA": ['railing looking out', 'across a plateau', 'up the coast', 'against the railing', 'in the courtyard', 'up the hill', 'above the gulf', 'outside the windows', 'long hallway toward', 'over the gulf', 'past the offices', 'through the windows', 'beside the river', 'past the waterfall', 'across the bridge', 'this long hallway', 'to a courtyard', 'and the courtyard', 'and behind me', 'down this embankment', 'towards the river', 'the hill up', 'courtyard was surrounded', 'in an alcove', 'onto the railing', 'along the coast', 'up the stairs', 'across the quadrangle', 'facing the beach', 'to the north', 'down the corridor', 'through the gates', 'over the embankment', 'onto the bridge', 'down that corridor', 'down the sidewalk', 'i looked across', 'path that jutted', 'through this door', 'the lagoon behind', 'down the embankment', 'on the railing', 'on the embankment', 'through the doors', 'on the windowsill', 'corridor out onto', 'the buildings beside', 'to the hallway', 'by that window', 'past the city', 'door behind me', 'to the south', 'off the coast', 'cross the bering', 'around the reef', 'behind me i', 'driveway and behind', 'against the windows', 'across the street', 'to the shoreline', 'lagoon behind the', 'on the sidewalk', 'hall past the', 'off the east', 'of the ravine', 'surrounded the city', 'in the window', 'southern shore of', 'in the distance', 'onto the sidewalk', 'i look across', 'behind us i', 'behind us there', 'on the cliff', 'over the river', 'toward the ocean', 'on that terrace', 'row of stalls', 'sidewalk in front', 'down the long', 'on the walls', 'door to the', 'by the window', 'outside my door', 'outside the door', 'across from me', 'on the eastern', 'the hall past', 'down the lagoon', 'in the forest', 'that window in', 'around me the', 'to the barrier', 'the gulf where', 'road in front', 'in the hallway', 'across the parking', 'in the colonnade', 'to the western', 'surrounded by rooms'],
    "PPA": ['in an alcove', 'on the stoop', 'past the offices', 'against the railing', 'on the windowsill', 'in the alley', 'to a courtyard', 'the copier room', 'in the courtyard', 'this long hallway', 'to the hallway', 'on a dock', 'in the hallway', 'long hallway toward', 'outside the windows', 'on that terrace', 'inside the hut', 'railing looking out', 'through the windows', 'down this embankment', 'on the subway', 'onto the subway', 'there were shelves', 'in my cubicle', 'a strip mall', 'on the sidewalk', 'in the colonnade', 'on the railing', 'into the basement', 'across the parking', 'a restaurant stoop', 'onto the railing', 'exit the subway', 'by the window', 'in that attic', 'was led upstairs', 'in the basement', 'the food court', 'and the courtyard', 'in the cafeteria', 'hall past the', 'into the parking', 'in the windowless', 'back room where', 'on my bed', 'down the sidewalk', 'contain strip malls', 'onto the sidewalk', 'the hall closet', 'at those cliffs'],

    "RSC_only": ['moved to chicago', 'drove from vermont', 'came to florida', 'here in manhattan', 'living in chicago',  'move to texas', 'leaving for france', 'back in manhattan', 'to boston to', 'went to boston', 'moved to vermont', 'geese in ohio', 'college in boston', 'in ohio', 'moved to brooklyn', 'normal suburban pittsburgh', 'moved to london', 'back in israel', 'to london to', 'come from israel', 'went to manchester', 'to columbus ohio', 'here in boston', 'i left vermont', 'from pittsburgh pennsylvania', 'in lower manhattan', 'hometown in texas', 'touring through europe', 'in warmer mexico', 'union in manhattan', 'suburban pittsburgh', 'moved to washington', 'was in boston', 'slacking in madison', 'chick from silverlake', 'heading to iraq', 'in chicago', 'in louisville kentucky', 'lived in hiroshima', 'in florida'],
    "OPA_only": ['eventually i forgave', 'push past it', 'eventually forgave', 'she eventually forgave', 'i forgave', 'to see ourselves', 'of myself which', 'means extending empathy', 'forgive and', 'i stopped myself', 'forgive and love', 'of the hurt', 'i rise above', 'i pushed myself', 'is and who', "'m hurt but", 'see ourselves and', 'the hurt', 'i persisted and', 'to forgive afterwards', 'was real to', 'comparing myself', 'looked in myself', 'selves which', 'around it and', 'self which translated', 'inside me that', 'of me which', 'overcome my ambivalence', 'to push myself', 'was also influenced', 'the side which', 'side which', 'independence and freedom'],
    "PPA_only": ['a garbage bag', 'that garbage bag', "'re throwing napkins", 'box of discarded', 'in sugar jars', 'their chew toys', 'those plastic containers', 'our dishwasher', 'skivvies toothbrush floss', 'of cheap beer', 'throwing napkins', 'a trash can', 'milk bottle tops', 'want a mcflurry', 'vomit smelling couch', 'grown napkins', 'my cheese sandwich', 'overpriced coffee shops', 'some lighter fluid', 'salad and stale', 'vomited a washpan', 'these brown paper'],

    "IPS": ['and behind me', 'against the railing', 'onto the railing', 'path that jutted', 'situated herself behind', 'above the gulf', 'door behind me', 'southern shore of', 'i looked across', 'along the edge', 'closed behind me', 'behind me and', 'across a plateau', 'on the railing', 'up behind me', 'leaning against the', 'towering above me', 'jutted into the', 'onto the bridge', 'and cut across', 'behind him and', 'and came around', 'front of us', 'up onto the', 'over the gulf', 'stood behind me', 'across the bridge', 'beside the river'],
    "sPMv": ['repeated her affirmation', 'said excuse me', 'asked i laughed', 'and goes hey', 'response was nah', 'hurry she exclaimed', 'said no i', 'just nodded yes', 'retorted rather loudly', 'was like hey', 'called her and', 'and said yes', 'and screams fuck', 'said uh hey', 'says sure and', 'says uh actually', 'was like hi', 'i said fine', 'said without hesitation', 'said well yes', 'says excuse me', 'i asked immediately', 'she yelled i', 'said mom mom', 'said did i', 'i said wow', 'i said shyly', 'asked her and', 'said okay okay', 'i sheepishly raise', 'which i responded', 'turned and said', 'then wrote yes', 'said yes i', 'whisper she said', 'was like mhm'],
    "EBA": ['arms around her', 'wraps his arms', 'hands gripped the', 'into my palm', 'hands into my', 'elbows on knees', 'grab his arms', 'his hands folded', 'into her arms', 'grabbed her hand', 'arms flailing', 'grabbed her legs', 'arm around my', 'grabbed their hands', 'lifted her up', 'put my arms', 'leaned his head', 'put his arms', 'shakes my hand', 'flying arms flailing', 'i leaned down', 'arms tighten around', 'her hands gripped', 'hand on his', 'my feet kicking', 'pinning my arms', 'held her hand', 'in a headlock', 'pressed my face', 'holds her hand', 'arms flailing holding', 'rubbing his head'],
    "OFA": ['and we talked', 'i even met', 'and so finally', 'one night my', 'one evening after', 'anyway the point', 'one summer my', 'weeks passed and', 'finally one day', 'then we talked', 'one night i', 'we chatted', 'and i talked', 'we talked and', 'talked and', 'so i texted', 'to my surprise', 'one afternoon when', 'i persisted and', 'and i finally', 'was watching television', 'i remember once', 'so one night', 'but anyway', 'and i met', 'when i finally', 'so i finally', 'on and on', 'son had finally', 'and i especially', 'so one day', 'and eventually i', 'we brace ourselves', 'so anyway', 'i was perusing', 'and it finally', 'later that day', 'so we finally', 'but anyhow', 'and as we', 'the day came', 'home one afternoon', 'then i finally', 'what fascinated me', 'and i vaguely', 'so i talked', 'once while i', 'was hanging out', 'i was reliving', 'but the most'],
}

rows = {
    'roi': rois,
    'expl': [explanations_clean[k] for k in rois],
    'top_ngrams_module_correct': [top_ngrams_clean[k] for k in rois],
    'stability_score': [stability_scores[k.split('_')[0]] for k in rois],
    'subject': [f'UT{subject}'] * len(rois),
    'voxel_nums': [rois_dict[k.split('_')[0]] for k in rois],
    'prompt_suffix': [explanation_avoid_suffixes[k] for k in rois],
}
rows = pd.DataFrame(rows)
rows.to_pickle(f'rows_roi_ut{subject.lower()}_may31.pkl')